In [ ]:
a#Import the neccessary packages.
import numpy as np
import pandas as pd
import pickle
import os
import sys
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, save_model
from tqdm import tqdm
from sklearn.manifold import TSNE
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D


2024-03-13 16:24:33.731720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 16:24:35.551483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Labeled
 Data Pool (train images, train labels)

In [ ]:
file_path = 'Data_Random_sampled(arr, lab)_3p.pkl'     #Importing random sampled 3% labelled dataset.
import pickle
with open(file_path, 'rb') as file:                    #loding the data in pickle file
    train_data = pickle.load(file)                     #storing teh data in list variable

print("len of the training data = ", len(train_data))
train_images,train_labels = zip(*train_data)           #extracting the numpy array image data and corresponding labels to two different list variables.
train_images = np.array(train_images)                  #converting to numpy array
train_labels = np.array(train_labels)                  #converting to numpy array
train_labels = to_categorical(train_labels)            #Converting labels to categorical values using one-hot encoding format
print(train_images.shape)                              #cross veriyfying the lenght of train images and train labels
print(train_labels.shape)                              #both lengths should be the same

len of the training data =  310
310
310
(310, 150, 150, 3)
(310, 4)


"file_path = '7023_085/img_arr_7023_(0.85)_12.pkl'\nwith open(file_path, 'rb') as file:\n    images = pickle.load(file)\n    \nfile_path = '7023_085/img_lab_7023_(0.85)_12.pkl'\nwith open(file_path, 'rb') as file:\n    labels = pickle.load(file)\n    \nprint(len(images))\nprint(len(labels))"

In [ ]:
model=load_model('DenseNet201_100_epochs.h5')  #Importing the pre-trained model which is trained using 3 % of lableled dataset
model.summary()                                #summary of the Model. Gives all the details of layers and input/output shapes

2024-03-13 16:24:40.311854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 16:24:40.827061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 16:24:40.827281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 1920)              18321984  
                                                                 
 dense (Dense)               (None, 128)               245888    
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 18568388 (70.83 MB)
Trainable params: 18339332 (69.96 MB)
Non-trainable params: 229056 (894.75 KB)
_________________________________________________________________


#UNCERTAINITY SCORES..


In [ ]:

def calculate_informativeness_scores(predictions):  #function to calculate informativeness and representativeness
    print("\nCalculating informativeness scores...")
    entropy = -np.sum(predictions * np.log(predictions + 1e-10), axis=1)     # Calculating entropy of predicted probabilities
    top_class_probabilities = np.max(predictions, axis=1)                    # picking the highes value in predicted probabilities
    second_top_class_probabilities = np.partition(predictions, -2)[:, -2]    # picking the second highes pvalue in predicted probabilities
    margin = top_class_probabilities - second_top_class_probabilities        # Calculating the margin
    weighted_sum = entropy + (1- margin)                                     #  sum of entropy and inverse of 1-margin (margin is inversely propotional to Entropy)
    print("\nInformativeness scores calculated successfully")
    return weighted_sum                                                      # returning the value of Informativeness

In [ ]:
file_path = 'Variances.pkl'                                                         #Loading the claclulated representativensess score for entire unlabaled data pool
with open(file_path, 'rb') as file:                                                 #opening file using python
    representativeness_scores = pickle.load(file)                                   #Storing the representaiveness scores in a list variable

print(f'The list has been loaded')
len(representativeness_scores)                                                      #length should equal to number of unlabled samples in the unlabled data pool
representativeness_scores=np.array(representativeness_scores)                       #converting to numpy arrays
representativeness_scores=representativeness_scores/max(representativeness_scores)  #normalising the representativeness scores

In [ ]:
#Calculating Combined uncertatinity scores (Parent step)
def calculate_combined_Uncertainity_scores(predictions,representativeness_scores):  #function calculate combined uncertainity scores
    informativeness_scores = calculate_informativeness_scores(predictions)          #retriving the informativeness scores
    informativeness_scores=informativeness_scores/max(informativeness_scores)       #retriving the representativeness scores
    alpha=0.5                                                                       #weight for informativeness
    beta=0.5                                                                        #weight for representativeness
    combined_scores=alpha*informativeness_scores+beta*representativeness_scores     #weighted sum
    print("\nCalculating combined scores...")
    print("The combined scores are calculated...")
    return combined_scores                                                          #returning the uncertaininty scores to the active learning funtion

#Intialising the active learning

In [ ]:
unlabeled_data_path='data(7023)_with_labels.pkl'     #Fetching the unlabeled data pool.
    print("Fetching Unlabeled data pool...")
with open(unlabeled_data_path, 'rb') as file:        #loading the unlabled data pool
    unlabeled_data = pickle.load(file)               #Storing the data in list varibale
print("\n\ncompleted fetching unlabeled data!!")

In [ ]:
active_cycle_number = 1                                                         #initialising active learning cycle number
while active_cycle_number<=25:                                                  #While loop for running through cycles
    print("\nIntialising ACTIVE LEARNING cycle no. ",active_cycle_number)       #prompt
    predictions = model.predict(X_arr)                                          #Feeding unlabled data pool to model and acquiring prediction probabilities
    combined_uncertainity_scores = calculate_combined_Uncertainity_scores(predictions,representativeness_scores) #retriving uncertainity scores
    print("Noralising the scores...")
    normalized_scores = combined_uncertainity_scores / max(combined_uncertainity_scores) #normalising thge uncertainity scores
    print("Selecting the most informative samples..")                           #prompt
    threshold = 0.95                                                            #setting iniatial threshold value
    while True:                                                                 #initialising "DYNAMIC THRESHOLDING"
        selected_indices = np.where(normalized_scores > threshold)[0]           #select all the normalised uncertaininty scores indices where the value is leass than current threshold
        if len(selected_indices) < 50:                                          #checking if required minimum number of samples are selected or not
            threshold -= 0.05                                                   #if not, we should decrease the value of threshold by 0.05
        else:                                                                   #else block ( which means, enough number of samples are selected.)
            break                                                               #break the dynamic threshold loop
    selected_data = [(X_arr[i], img_labels[i]) for i in selected_indices]       #using the selected indices above, we are fetching the data from unlabled pool data of image arrays
    print("\nSelection of informative samples is complete!..")                  #prompt
    #GUI WILL COME HERE
    print("\nNumber of Informativeness samples selected in the current loop = ",len(selected_data)) #PROMPT
    selected_data_tuples = [(tuple(X), img_label) for X, img_label in selected_data] #converting the extracted dat into tuples of labels and images
    print("The number of selected samples for retraining the model = ", len(selected_data)) #prompt
    print("\nPress Enter to continue training model...!!")                      #prompt
    selected_images, selected_labels =zip(*selected_data)                       #separating images and labels into two different lists
    print("\nAppending the sampled data pool to the existing labeled data..")   #prompt

    # Create a mask to filter out selected indices
    '''mask = np.ones(len(X_arr), dtype=bool)                                   #to be verified
    mask[selected_indices] = False                                              #to be verified
    X_arr = X_arr[mask]                                                         #to be verified
    representativeness_scores = representativeness_scores[mask]                 #to be verified
    print("Reduced - Number of samples in unlabled data pool: ",len(X_arr))     #to be verified'''

    images = images + selected_images                                           #appending new selected images to exisiting labeled data pool
    labels = labels + selected_labels                                           # appending corresponding labels to exisiting labeled data pool


    train_images= np.array(images)                                              #converting to numpy arrays
    train_images= train_images.reshape(-1,150,150,3)                            #reshaping the new set to fit into models input shape
    train_labels = np.array(labels)                                             #converting to numpy arrays
    train_labels = to_categorical(labels, num_classes=4)                        #converting labels as categorical values using onehot encoding method.


    print("Now, The total number of data samples for retraining the model is = ",len(train_images)) #prompt
    print("\nThe total percentage of data used for current cycle = ", (len(train_images)/10357))    #prompt

    #Training the active learning model
    print("\nPreparing for training a model..")
    #spliting data for training and testing using Train_test_split function in python
    X_train, X_test, y_train, y_test = train_test_split(train_images,train_labels, test_size=0.2,random_state=44)
    #data agumentation
    data_aug = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20, zoom_range=0.2,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.1, fill_mode="nearest")
    print("\nTraining the Model...")
    #Training the model existing model with new data with 50 epochs and 32 batch size
    history = model.fit(data_aug.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=50)
    y_pred=model.predict(X_test)                                                #testing the model
    print("\nTraining Completed!!..")                                           #prompt
    print("\n\nTest Results .....")                                             #prompt
    pred = np.argmax(y_pred,axis=1)                                             #prediction classes extraction using argmax function to calculate accuracy
    original = np.argmax(y_test,axis=1)                                         #taking the original labels of test dataset to test
    print(classification_report(original,pred))                                 #printing classification report using predicted class labels and original class labels

    #training accuracy and validation accuracy graph
    print('\n\nTraining accuracy and validation accuracy graph ..')
    epochs = range(1, len(history.history['accuracy']) + 1)
    plt.plot(epochs, history.history['accuracy'], 'r', label='Accuracy of Training data')
    plt.plot(epochs, history.history['val_accuracy'], 'b', label='Accuracy of Validation data')
    plt.title('Training vs validation accuracy')
    plt.legend(loc=0)
    plt.figure()
    plt.show()


    #training loss and validation loss graph
    print('\n\nTraining loss and validation loss graph....')
    epochs = range(1, len(history.history['accuracy']) + 1)
    plt.plot(epochs, history.history['loss'], 'r', label='Loss of Training data')
    plt.plot(epochs, history.history['val_loss'], 'b', label='Loss of Validation data')
    plt.title('Training vs validation loss')
    plt.legend(loc=0)
    plt.figure()
    plt.show()


    #Accuracy printing
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(original, pred)
    print("\nAccuracy: ", accuracy)


    #Saving The model
    model.save(os.path.join("D_095", f'model_(D)_{active_cycle_number}.h5'))    #save the model for each iteration.
    active_cycle_number += 1                                                    #proceeding to next active learning cycle.